## Team 2022-180's research project front-end with back-end

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import string
import re
import googletrans
from googletrans import Translator
from googletrans import LANGUAGES
translator = Translator()

from flask import Flask, render_template, redirect, url_for, request 

app = Flask(__name__)

@app.route("/")
@app.route("/home")
def home():
    return render_template("index.html")
  
@app.route("/tanglish-translation")
def process_01():
    return render_template("process_01.html")

@app.route("/sentimental-analysis")
def process_02():
    return render_template("process_02.html")

@app.route("/genre-analysis")
def process_03():
    return render_template("process_03.html")

@app.route("/add-latest-comment")
def process_04():
    return render_template("process_04.html")

@app.route("/about")
def about():
    return render_template("about.html")

#---------------------component-01-------------------------------------------------------------(start)
translated_words = []
src_language = []
dest_language = []
translated_tamil_words = []
get_Letters = []
encode = []
encode_list = []

# dictionaries for encoding & encoding
d_tamil = {"அ":"0", "ஆ":"1", "இ":"2", "ஈ":"3", "உ":"4", "ஊ":"5", "எ":"6", "ஏ":"7", "ஐ":"8", "ஒ":"9", "ஓ":"!", "ஔ":"@",
                    "க":"#", "ங":"$", "ச":"%", "ஜ":"^", "ஞ":"&", "ட":"*", "ண":"-", "த":"+", "ந":"=", "ன":"a", "ப":"b", "ம":"c","ய":"d", "ர":"e", "ற":"f", "ல":"g", "ள":"h", "ழ":"i", "வ":"j", "ஷ":"k", "ஸ":"l", "ஹ":"m",
                    "்":"n", "ா":"o", "ி":"p", "ீ":"q", "ு":"r", "ூ":"s", "ெ":"t", "ே":"u", "ை":"v", "ொ":"w", "ோ":"x", "ௌ":"y", "ௗ":"~", "ஃ":"z"
                   }

d_english = {"0":"A", "1":"A", "2":"I", "3":"I", "4":"U", "5":"U", "6":"E", "7":"E", "8":"AI", "9":"O", "!":"O", "@":"AU",
                        "#":"K", "$":"N", "%":"S", "^":"J", "&":"N", "*":"D", "-":"N", "+":"TH", "=":"N", "a":"N", "b":"P", "c":"M", "d":"Y", "e":"r", "f":"r", "g":"L", "h":"L", "i":"L", "j":"V", "k":"S", "l":"S", "m":"H",
                        "n":"", "o":"a", "p":"i", "q":"i", "r":"u", "s":"u", "t":"e", "u":"e", "v":"ai", "w":"o", "x":"o", "y":"au", "~":"", "z":"H"        
                    }


punctuation = '''!()@#&:;🤜🤛🎂😎💚...❤/%?\{}$,-😂🙏*👇🔥😢👹💘😊👍👎👊😣🤕😈😄☠😭1234567890😀💗🥰😍💯🤩♥️🤘👈✌️🙂💪✊🏿😘🥳🤟😫😊💖️🌹👌🥳🤗🤮🤢🎬🥳🤯😇🤣🏽😃💥🤔'''
def remove_punctuation(cmt):
    cmt_nonpunct = "".join([ char for char in cmt if char not in punctuation])                               
    return cmt_nonpunct

def tokenize(cmt):
    tokens = re.split('\W+', cmt)
    return tokens

remove_words = []
remove_words.append('httpswwwyoutubecomwatchv')
remove_words.append('yjlRWSiampt')
remove_words.append('href')
remove_words.append('br')
remove_words.append('Bar')
remove_words.append('Kahatiti')
remove_words.append('ms')
remove_words.append('a')
remove_words.append('the')
remove_words.append('tha')

def remove_stop_words(cmt_tokenize):
    clean_cmt = [word for word in cmt_tokenize if word not in remove_words]
    return clean_cmt

def merge_tokens(cmt):
    merge_token = " ".join(cmt)                                                                                                          
    return merge_token


# import data-set
@app.route('/uploader', methods=['GET', 'POST'])
def file_import():
    if request.method == 'POST':
        global get_file
        get_file = request.form['upload-excel']
        if get_file != '':
            movie_comments = pd.read_excel(get_file)
            return render_template("process_01.html", data=movie_comments.to_html(index=False).replace('<th>', '<th style="text-align:center; color:#a8031f; font-size:20px ">'))
            
        return render_template("process_01.html", error="Please Import the File!") # else...
    

# remove punctuation
@app.route("/cleaningComments")
def cleaning_process():
#     print(get_file)
    if get_file != '':
        data1 = pd.read_excel(get_file)
        data1['Cleaned_Comments'] = data1['Original_Comments'].apply(lambda x: remove_punctuation(x)) # remove punctuation marks
        global Cleaned_Comments
        Cleaned_Comments = data1['Cleaned_Comments'] 
        return render_template("process_01.html", cleanedComments=data1.to_html(index=False).replace('<th>', '<th style="text-align:center; color:#a8031f; font-size:20px ">'))
    return render_template("process_01.html", error1="There are no dataset files!")


# predict language & translate to English language
@app.route("/predictLanguage")
def predict_language():
    if get_file != '':
        data2 = pd.read_excel(get_file)
        for element in Cleaned_Comments:
            translated_words.append(translator.translate(element).text) # translate to English language
            src_language.append(LANGUAGES[ translator.translate(element).src] ) # predict source language
            dest_language.append(LANGUAGES[ translator.translate(element).dest] ) # predict destination language
        data2['Cleaned_Comments'] = Cleaned_Comments # add with data2
        data2['Source_Language'] = src_language
        data2['destination_Language'] = dest_language
        data2['English_Comments'] = translated_words
        
        # store as global variable
        global english_comments
        english_comments = data2['English_Comments']
        global Source_Language
        Source_Language = data2['Source_Language']
        global destination_Language
        destination_Language = data2['destination_Language']
        
        return render_template("process_01.html", translation=data2.to_html(index=False).replace('<th>', '<th style="text-align:center; color:#a8031f; font-size:20px ">'))
    
    return render_template("process_01.html", error2="There are no dataset files!")


# basic NLTK process
@app.route("/basicNLTK")
def basic_Process():
    if get_file != '':
        data3 = pd.read_excel(get_file)
        
        data3['Cleaned_Comments'] = Cleaned_Comments # add with data3
        data3['Source_Language'] = Source_Language # add with data3
        data3['destination_Language'] = destination_Language # add with data3        
        data3['English_Comments'] = english_comments # add with data3
        
        data3['tokens'] = english_comments.apply(lambda x: tokenize(x)) # tokenization
        data3['cleaned_tokens'] = data3['tokens'].apply(lambda x: remove_stop_words(x)) # remove unwanted token
        data3['merge_tokens'] = data3['cleaned_tokens'].apply(lambda x: merge_tokens(x)) # merge token
        
        global tokens, cleaned_tokens, merge_tokens
        tokens = data3['tokens']
        cleaned_tokens = data3['cleaned_tokens']
        merge_tokens = data3['merge_tokens'] 
        
        return render_template("process_01.html", basicNLTK=data3.to_html(index=False).replace('<th>', '<th style="text-align:center; color:#a8031f; font-size:20px ">'))
    
    return render_template("process_01.html", error3="There are no dataset files!")


# translate to Tamil language
@app.route("/translateToTamil")
def translate_tamil():
    if get_file != '':
        data4 = pd.read_excel(get_file)
        for element in merge_tokens:
            translated_tamil_words.append(translator.translate(element, dest="ta").text) # translate to Tamil language
            
        data4['Cleaned_Comments'] = Cleaned_Comments # add with data4
        data4['Source_Language'] = Source_Language # add with data4
        data4['destination_Language'] = destination_Language # add with data4        
        data4['English_Comments'] = english_comments # add with data4
        data4['tokens'] = tokens # add with data4
        data4['cleaned_tokens'] = cleaned_tokens # add with data4
        data4['merge_tokens'] = merge_tokens # add with data4
        data4['Tamil_Comments'] = translated_tamil_words # result of translate to tamil language
        
        global tamil_comments
        tamil_comments = data4['Tamil_Comments']
        
        return render_template("process_01.html", traslateToTamil=data4.to_html(index=False).replace('<th>', '<th style="text-align:center; color:#a8031f; font-size:20px ">'))
    
    return render_template("process_01.html", error4="There are no dataset files!")


# encoding & decoding process & get Tanglish as output 
@app.route("/tanglish")
def get_tanglish():
    if get_file != '':
        data5 = pd.read_excel(get_file)
            
        data5['Cleaned_Comments'] = Cleaned_Comments 
        data5['Source_Language'] = Source_Language 
        data5['destination_Language'] = destination_Language     
        data5['English_Comments'] = english_comments 
        data5['tokens'] = tokens
        data5['cleaned_tokens'] = cleaned_tokens
        data5['merge_tokens'] = merge_tokens 
        data5['Tamil_Comments'] = tamil_comments 
        
        # encoding & decoding process
        for tamil_cmt in tamil_comments:
        # print(tamil_cmt)
            for char in tamil_cmt:
                get_Letters.append(char) 
            get_Letters.append("\n ")
        TL = ''.join(get_Letters)
        encode = re.sub('({})'.format( '|'.join(map(re.escape, dict_tamil.keys() ))), lambda m: dict_tamil[m.group()], TL)
        # print(encode)
        for c in encode:
            encode_list.append(c)
        # print(encode_list)

        get_result = ""
        for index, val in enumerate(encode): #----Validation for tamil-letters----
            if val=='#' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='$' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='%' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='^' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='&' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='*' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='-' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='+' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='=' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='a' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='b' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='c' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='d' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='e' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='f' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='g' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='h' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='i' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='j' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='k' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='l' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='m' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

        # print(get_result)
        encode = ''.join(get_result)
        # print("========= En-Code =========", "\n", "\n", encode) 

        step_2 = re.sub('({})'.format('|'.join(map(re.escape, dict_english.keys() ))), lambda m: dict_english[m.group()], encode)
        decode = step_2.capitalize()
        # print("========= De-Code(Tamil-English Code-mix Data) =========", "\n", "\n", decode) 

        store1 = []
        store2 = []
        store1.append( encode.title().rstrip().split('\n'))
        store2.append( decode.title().rstrip().split('\n'))
        # print(store1)
        # print(store2)

        temp1 = []
        temp2 = []
        for front_index in store1:
            for second_index in front_index:
                temp1.append(second_index)   
        #     print(temp1)

        for front_index in store2:
            for second_index in front_index:
                temp2.append(second_index)   
        #     print(temp2)

        # store into the data-set
        data5['encode'] = temp1 
        data5['decode(Tanglish_Comment)'] = temp2 
        
        global encodes, decodes
        encodes = data5['encode']
        decodes = data5['decode(Tanglish_Comment)']
        
        return render_template("process_01.html", tanglishComments=data5.to_html(index=False).replace('<th>', '<th style="text-align:center; color:#a8031f; font-size:20px ">'))
    
    return render_template("process_01.html", error5="There are no dataset files!")

# Save Tanglish-comments file as an Excel.
@app.route("/save") 
def save_file():
    if get_file != '':
        data6 = pd.read_excel(get_file)
            
        data6['Cleaned_Comments'] = Cleaned_Comments # add with data6
        data6['Source_Language'] = Source_Language # add with data6
        data6['destination_Language'] = destination_Language # add with data6       
        data6['English_Comments'] = english_comments # add with data6
        data6['tokens'] = tokens # add with data6
        data6['cleaned_tokens'] = cleaned_tokens # add with data6
        data6['merge_tokens'] = merge_tokens # add with data6
        data6['Tamil_Comments'] = tamil_comments # add with data6
        data6['encode'] = encodes # add with data6
        data6['decode(Tanglish_Comment)'] = decodes # add with data6
        
        df = pd.DataFrame(data6)

        writer = pd.ExcelWriter('process_01 output - Tanglish_Comments.xlsx', engine='xlsxwriter')
        df.to_excel(writer, sheet_name="tanglish_translator")
        writer.save()
        
        return render_template("process_01.html", msg_01="File Successfully Saved.")
    
    return render_template("process_01.html", msg_02="Error!!!")
#---------------------component-01---------------------------------------------------------------(end)



#---------------------component-02-----------------------------------------------------------(start)
positive_list = ['nanraka', 'alakana', 'siranthathu', 'verupadda', 'virumpukiren', 'olukkamana', 'nalla', 'sirantha', 'suppar', 'maas', 'rasikan', 'geya', 'veramathi', 'nalla', 'senjudar', 'nallam', 'paarkalaam', 'senjudar', 'senjudanga', 'neruppu', 'mersal', 'next level', 'level', 'vera kaddam', 'verelevel']
negative_list = ['virayam', 'kothikalan', 'keli', 'kalivu', 'kanravi', 'kevalam', 'asinkam', 'kevalamana', 'asinkamana', 'bore', 'boomer', 'beep', 'nallave illa', 'nalla illa', 'poor', 'mayiru', 'bad', 'failure', 'fail', 'sleep']

def token(tok):
    token = re.split('\W+', tok)
    return token

@app.route('/uploadTanglish', methods=['GET', 'POST'])
def tanglish_import():
    if request.method == 'POST':
        global file1
        file1 = request.form['upload-tanglish']
        if file1 != '':
            tanglish_comments = pd.read_excel(file1)
            # use this line, if you want to show specific column....
            # data_table = pd.concat([ tanglish_comments[tanglish_comments.columns[11:]] ], axis=1)
            data_table = tanglish_comments
            return render_template("process_02.html",tanglish_dataset=data_table.to_html(index=False).replace('<th>', '<th style="text-align:center; color:#a8031f; font-size:20px ">'))
            
        return render_template("process_02.html", error="Please Import the File!")

@app.route("/tokenization")
def tokenization():
    if file1 != '':
        tang_data = pd.read_excel(file1)
        
        tang_data['Tokens'] = tang_data['decode(Tanglish_Comment)'].apply(lambda x: tokz(x)) # tokenization
        global token
        token = tang_data['Tokens']
#         data_table = pd.concat([ tang_data[tang_data.columns[11:]] ], axis=1)
        data_table = tang_data
        
        return render_template("process_02.html", tokens=data_table.to_html(index=False).replace('<th>', '<th style="text-align:center; color:#a8031f; font-size:20px ">'))
    
    return render_template("process_02.html", err1="There are no dataset files!")



@app.route("/count")
def get_count():
    global possitive, negative;
    
    if file1 != '':
        tang_data = pd.read_excel(file1)
        tang_data['Tokens'] = token #add tokens with tang_data1 files
        
        positive_comments_count = 0;
        negative_comments_count = 0;
        
        for cmt in tang_data['Tokens']:  
            try:  
                for word in cmt:
                    if word.lower() in positive_list:
                        while word.lower() not in positive_list:
                            continue
                        positive_comments_count += 1 
#                         print(word)
                        break

                for word in cmt:
                    if word.lower() in negative_list:
                        while word.lower() not in negative_list:
                            continue
                        negative_comments_count += 1
#                         print(word)
                        break
            except:
                pass
            
        status = 'Positive Comments: ', positive_comments_count,  ' Negative Comments:', negative_comments_count
        pos = positive_comments_count;
        neg = negative_comments_count;
        
        positive_rating = pos/(pos + neg)
        positve_percentage = round(positive_rating*100, 2)

        negative_rating = neg/(pos + neg)
        negative_percentage = round(negative_rating*100, 2)
        
        possitive = positve_percentage;
        negative = negative_percentage;
        
        return render_template("process_02.html", counts=status)
    return render_template("process_02.html", err2="Counting failed!")
    

@app.route("/percentage")
def count_percentage():
    global pos_rating, neg_rating;
    
    if possitive != 0.0 and negative != 0.0:    
        pos_rating = possitive
        neg_rating = negative
        
        status = 'Positive percentage: ', pos_rating , '%', 'Negative percentage:', neg_rating , '%'
        
        return render_template("process_02.html", percentage = status)
    return render_template("process_02.html", err3="There are no comments!")

@app.route("/decision")
def final_decision():
    if pos_rating != 0.0 and neg_rating != 0.0:    
        
        if pos_rating > neg_rating:
            status = 'This movie has more "positive" review than negative review. So, It is a Good Movie!'
        else:
            status = 'This movie has more "negative" review than positive review. So, It is a Average Movie!'
        
        return render_template("process_02.html", decision = status)
    return render_template("process_02.html", err4="There are no comments percentage!")
#---------------------component-02--------------------------------------------------------------(end)



#---------------------component-03-----------------------------------------------------------------(start)
comedy_list = ['sirippu', 'siriththan', 'comedy']
action_list = ['sandai','sandaik', 'sandaikal', 'sandaikasikal', 'thi', 'Nerupu', 'Kothikalan', 'Athiradi', 'kathaikkalam']
love_list = ['kathal', 'lovers', 'kathalarkal', 'kathalar', 'kaviyam', 'virumpukiren', 'olukkamana', 'alakana']
horror_list = ['payama', 'payanthudan', 'thikil', 'pei']

def tokz(tok):
    token = re.split('\W+', tok)
    return token

@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        global file2
        file2 = request.form['upload-tanglish']
        if file2 != '':
            uploaded_dataset = pd.read_excel(file2)
#             data_table = pd.concat([ uploaded_dataset[uploaded_dataset.columns[11:]] ], axis=1)
            data_table = uploaded_dataset
            return render_template("process_03.html", dataset=data_table.to_html(index=False).replace('<th>', '<th style="text-align:center; color:#a8031f; font-size:20px ">'))
            
        return render_template("process_03.html", error="Please Import the File!")
    

@app.route("/tok")
def make_tokens():
    if file2 != '':
        tanglish = pd.read_excel(file2)
        
        tanglish['Tokens'] = tanglish['decode(Tanglish_Comment)'].apply(lambda x: tokz(x)) # tokenization
        global tok
        tok = tanglish['Tokens']
#         data_table = pd.concat([ tanglish[tanglish.columns[11:]] ], axis=1)
        data_table = tanglish
        return render_template("process_03.html", tokens=data_table.to_html(index=False).replace('<th>', '<th style="text-align:center; color:#a8031f; font-size:20px ">'))
    
    return render_template("process_03.html", err1="There are no dataset files!")



@app.route("/genres")
def get_genres():
    global comedy, love, horror, action;
    
    if file2 != '':
        tanglish = pd.read_excel(file2)
        tanglish['Tokens'] = tok 
        
        comedy_comments_count = 0;
        action_comments_count = 0;
        love_comments_count = 0;
        horror_comments_count = 0;

        for cmt in tanglish['Tokens']:  
            try:  
                for word in cmt:
                    if word.lower() in comedy_list:
                        while word.lower() not in comedy_list:
                            continue
                        comedy_comments_count += 1 
                        # print(word)
                        break

                for word in cmt:
                    if word.lower() in action_list:
                        while word.lower() not in action_list:
                            continue
                        action_comments_count += 1
                        # print(word)
                        break

                for word in cmt:
                    if word.lower() in love_list:
                        while word.lower() not in love_list:
                            continue
                        love_comments_count += 1
                        # print(word)
                        break

                for word in cmt:
                    if word.lower() in horror_list:
                        while word.lower() not in horror_list:
                            continue
                        horror_comments_count += 1
                        # print(word)
                        break
            except:
                pass

        status = 'Comedy Comments: ', comedy_comments_count, 'Action Comments: ', action_comments_count, 'Love Comments: ', love_comments_count, 'Horror Comments: ', horror_comments_count;
        c = comedy_comments_count;
        a = action_comments_count;
        l = love_comments_count;
        h = horror_comments_count;

        comedy_rating = c/(c+a+l+h)
        comedy_percentage = round(comedy_rating*100, 2)

        action_rating = a/(c+a+l+h)
        action_percentage = round(action_rating*100, 2)
        
        love_rating = l/(c+a+l+h)
        love_percentage = round(love_rating*100, 2)
        
        horror_rating = h/(c+a+l+h)
        horror_percentage = round(horror_rating*100, 2)

        comedy = comedy_percentage;
        action = action_percentage;
        love = love_percentage;
        horror = horror_percentage;

        return render_template("process_03.html", genres=status)
    return render_template("process_03.html", err2="Counting failed!")

@app.route("/genre_percentage")
def genre_percent():      
    status = 'Comedy percentage: ', comedy ,'%','action percentage:', action , '%','love percentage:', love , '%','horror percentage:', horror , '%'
    return render_template("process_03.html", percentage = status)


@app.route("/final_decision")
def make_decision(): 
    resu = 'No decision...'
    
# ----Testing purpose----   
#     comedy = 40
#     action = 40
#     love = 10
#     horror = 5
    
    if comedy>action and comedy>love and comedy>horror:
        resu = 'It is a Comedy genre movie!'
    elif action>comedy and action>love and action>horror:
        resu = 'It is a Action genre movie!'
    elif love>comedy and love>action and love>horror:
        resu = 'It is a Love genre movie!'
    elif horror>comedy and horror>action and horror>love:
        resu = 'It is a Horror genre movie!'  
    elif (comedy==action) and (comedy>love and comedy>horror):
        resu = 'It is a Comedy & Action genres movie!' 
    elif (comedy==love) and (comedy>action and comedy>horror):
        resu = 'It is a Comedy & Love genres movie!' 
    elif (comedy==horror) and (comedy>love and comedy>action):
        resu = 'It is a Comedy & Horror genres movie!'
    elif (action==love) and (action>comedy and action>horror):
        resu = 'It is a Action & Love genres movie!'
    elif (action==horror) and (action>comedy and action>love):
        resu = 'It is a Action & Horror genres movie!'
    elif (love==horror) and (love>comedy and love>action):
        resu = 'It is a Love & Horror genres movie!'
    return render_template("process_03.html", deci = resu)

#---------------------component-03-------------------------------------------------------------------(end)



#---------------------component-04-------------------------------------------------------------------(start)

punctuation = '''!()@#&:;"🤜🤛🎂😎💚...❤/%?\{}$,-😂🙏*👇🔥😢👹💘😊👍👎👊😣🤕😈😄☠😭1234567890😀💗🥰😍💯🤩♥️🤘👈✌️🙂💪✊🏿😘🥳🤟😫😊💖️🌹👌🥳🤗🤮🤢🎬🥳🤯😇🤣🏽😃💥🤔'''

# Tamil-English dictionary
dict_tamil = {"அ":"0", "ஆ":"1", "இ":"2", "ஈ":"3", "உ":"4", "ஊ":"5", "எ":"6", "ஏ":"7", "ஐ":"8", "ஒ":"9", "ஓ":"!", "ஔ":"@",
                    "க":"#", "ங":"$", "ச":"%", "ஜ":"^", "ஞ":"&", "ட":"*", "ண":"-", "த":"+", "ந":"=", "ன":"a", "ப":"b", "ம":"c","ய":"d", "ர":"e", "ற":"f", "ல":"g", "ள":"h", "ழ":"i", "வ":"j", "ஷ":"k", "ஸ":"l", "ஹ":"m",
                    "்":"n", "ா":"o", "ி":"p", "ீ":"q", "ு":"r", "ூ":"s", "ெ":"t", "ே":"u", "ை":"v", "ொ":"w", "ோ":"x", "ௌ":"y", "ௗ":"~", "ஃ":"z"
                   }

dict_english = {"0":"A", "1":"A", "2":"I", "3":"I", "4":"U", "5":"U", "6":"E", "7":"E", "8":"AI", "9":"O", "!":"O", "@":"AU",
                        "#":"K", "$":"N", "%":"S", "^":"J", "&":"N", "*":"D", "-":"N", "+":"TH", "=":"N", "a":"N", "b":"P", "c":"M", "d":"Y", "e":"r", "f":"r", "g":"L", "h":"L", "i":"L", "j":"V", "k":"S", "l":"S", "m":"H",
                        "n":"", "o":"a", "p":"i", "q":"i", "r":"u", "s":"u", "t":"e", "u":"e", "v":"ai", "w":"o", "x":"o", "y":"au", "~":"", "z":"H"        
                    }


@app.route('/write', methods=['GET', 'POST'])
def submit_cmt():
    if request.method == 'POST':
        global user_cmts
        user_cmts = request.form['user_cmt']
        if user_cmts != '':
            return render_template("process_04.html", wrote_1=user_cmts);
        
        return render_template("process_04.html", error1="Please Type Your Comments!");
    

@app.route("/TransalteToTanglish")
def trans_to_tang():
    if user_cmts != '':
        global english_cmt, tamil_cmt, decode, encode;
        
        # remove punctuation only
        pure_cmt = [''.join(c for c in user_cmts if c not in punctuation)];
        
        # translate to english
        for a in pure_cmt:
            english_cmt = translator.translate(a, dest="en").text;

        # translate to tamil   
        tamil_cmt = translator.translate(english_cmt, dest="tamil").text;
        
        # translate to tanglish
        # convert to tamil-english code-mix data, apply algorithm for en-code and de-code
        get_Letters = []
        encode = []
        encode_list = []

        for tc in tamil_cmt:
        #     print(tamil_cmt)
            get_Letters.append(tc) 
        TL = ''.join(get_Letters)
        encode = re.sub('({})'.format( '|'.join(map(re.escape, dict_tamil.keys() ))), lambda m: dict_tamil[m.group()], TL)
        # print(encode)
        for c in encode:
            encode_list.append(c)
        # print(encode_list)

        get_result = ""

        for index, val in enumerate(encode):
            if val=='#' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='$' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='%' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='^' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='&' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='*' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='-' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='+' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='=' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='a' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='b' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='c' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='d' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='e' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='f' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='g' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='h' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='i' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='j' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='k' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='l' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

            elif val=='m' and encode[index+1]!='n' and encode[index+1]!='o' and encode[index+1]!='p' and encode[index+1]!='q' and encode[index+1]!='r' and encode[index+1]!='s' and encode[index+1]!='t' and encode[index+1]!='u' and encode[index+1]!='v' and encode[index+1]!='w' and encode[index+1]!='x' and encode[index+1]!='y':
                set_value = (val+dict_tamil["ா"])
                encode_list[index] = set_value
                get_result = encode_list

        # print(get_result)
        encode = ''.join(get_result)

        step_2 = re.sub('({})'.format('|'.join(map(re.escape, dict_english.keys() ))), lambda m: dict_english[m.group()], encode)
        decode = step_2.capitalize()

        status = 'Code-Mixed data- ', encode , ' Tanglish data- ', decode
        
        return render_template("process_04.html", wrote_2=status);
    return render_template("process_04.html", error2="There are no comment available...")


@app.route("/AddWithDataset")
def add_with_dataset():
    if decode != '':
        
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        data_set = pd.read_excel(r'C:\Users\Sajee\Desktop\Research-2023 group\2023-137_Project_PP2\process_01 output - Tanglish_Comments.xlsx') # change your pc path..
        data_table = pd.concat([ data_set[data_set.columns[11:]] ], axis=1) # get only tanglish column(11)... 
#         print(data_table)


        # data spilt and store into the list
        store = []
        store.append( decode.title().rstrip().split('\n'))

        temp = []
        for front_index in store:
            for second_index in front_index:
                temp.append(second_index)   
#         print(temp)


        # create new column for latest tanglish data
        new_column = pd.DataFrame({ "latest_tanglish": temp})

        # merge 2 columns data
        latest_data_set = data_table["decode(Tanglish_Comment)"].append(new_column["latest_tanglish"]).reset_index(drop=True)

        #  store dataset
        store_data = pd.DataFrame(latest_data_set)
        
#         print(data_table) # before add latest Tanglish cmt....
#         print("-----------------------------------------------------")
#         print(store_data) # after added latest Tanglish cmt....

        writer = pd.ExcelWriter('result-latest_tanglish_dataset.xlsx', engine='xlsxwriter')
        store_data.to_excel(writer, sheet_name="tanglish_latest_list")

        # rename column name 
        renamed_dataset = store_data.rename(mapper={ 0: 'decode(Tanglish_Comment)' }, axis='columns') 
        renamed_dataset.to_excel(writer, sheet_name="tanglish_latest_list")

        # store the latest dataset
        response = writer.save() 
        print("latest data saved :" , response)
        
        return render_template("process_04.html", wrote_03="File saved successfully!")
    return render_template("process_04.html", error3="Error!!!")
    



#---------------------component-04-------------------------------------------------------------------(end)


if __name__=='__main__':
    app.run()
   



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Oct/2023 08:16:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Oct/2023 08:16:21] "GET /static/customStyle.css HTTP/1.1" 304 -
127.0.0.1 - - [30/Oct/2023 08:16:21] "GET /static/carousel_05.jpg HTTP/1.1" 304 -
127.0.0.1 - - [30/Oct/2023 08:16:21] "GET /static/carousel_03.jpg HTTP/1.1" 304 -
127.0.0.1 - - [30/Oct/2023 08:16:21] "GET /static/carousel_02.jpg HTTP/1.1" 304 -
127.0.0.1 - - [30/Oct/2023 08:16:21] "GET /static/carousel_04.jpg HTTP/1.1" 304 -
127.0.0.1 - - [30/Oct/2023 08:16:21] "GET /static/carousel_06.jpg HTTP/1.1" 304 -
127.0.0.1 - - [30/Oct/2023 08:16:21] "GET /static/img_01.jpg HTTP/1.1" 304 -
127.0.0.1 - - [30/Oct/2023 08:16:21] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Oct/2023 08:16:24] "GET /add-latest-comment HTTP/1.1" 200 -
127.0.0.1 - - [30/Oct/2023 08:16:24] "GET /static/bac_04.jpg HTTP/1.1" 304 -
127.0.0.1 - - [30/Oct/2023 08:16:24] "GET /static/Jailer.mp4 HTTP/1.1" 206 -
127.0.0.

127.0.0.1 - - [30/Oct/2023 08:21:00] "GET /static/Jailer.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Oct/2023 08:21:00] "GET /static/Jailer.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Oct/2023 08:21:00] "GET /static/Jailer.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Oct/2023 08:21:09] "POST /write HTTP/1.1" 200 -
127.0.0.1 - - [30/Oct/2023 08:21:09] "GET /static/Jailer.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Oct/2023 08:21:12] "GET /TransalteToTanglish HTTP/1.1" 200 -
127.0.0.1 - - [30/Oct/2023 08:21:13] "GET /static/Jailer.mp4 HTTP/1.1" 206 -
[2023-10-30 08:21:15,596] ERROR in app: Exception on /AddWithDataset [GET]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle